In [1]:
import sys
import snowflake.connector as sf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
ctx =sf.connect(
    user='mgupta1@chewy.com',
    autocommit = 0,
    authenticator= "externalbrowser",
    account='chewy.us-east-1'
    )

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://chewy.okta.com/app/snowflake/exkmzig18yW58r3j00x7/sso/saml?SAMLRequest=lZLRbtowFIZfJfKuEzsBWmYBVQqrQKIdhbBq3HnJAdwkdubjNLCnnwml6i5aaXeR8%2F32Z%2F9ncHMoC%2B8FDEqthiQMGPFApTqTajck6%2BTO7xMPrVCZKLSCITkCkpvRAEVZVDyu7V4t4XcNaD23kULe%2FhiS2iiuBUrkSpSA3KZ8Fd%2FPeRQwXhltdaoL8i7yeUIggrHO8BLJUDq9vbUVp7RpmqDpBNrsaMQYo%2BwrddQJ%2BXLhD%2B5OH%2FAhZd0T7wiHL17dbqU6P8FnWr%2FOEPJpkiz8xfdVQrz4ojrWCusSzArMi0xhvZyfBdAZjKffnn4GNfog0PphgEo320LkkOqyqq3bM3BfdAsZLfROumvPJkNS5TKbbXZiuk83j%2FFkWe7jvCvS%2BhrunytIottyIxZizY5xOM9FNyXej0uv0anXGWINM3Vq07olFnV91vM7LAkj3ot41As6V1cb4k1cm1IJ2yYvyukemmOgcytaNVFV9M2awiEv%2F8hd2D8%2B9fqm88zY4ZoianqqlpynhbfHm9H%2FvcGAvs%2B%2Bjt2Da2I2WehCpkfvTptS2I%2BLCoOwXZGZv21RDqWQRZxlBhBdYUWhm7EBYd10W1MDoaPzqf%2FO9%2Bgv&RelayState=56227 

In [3]:
sql = """ 
SELECT * FROM MRCH_SANDBOX.LOYALTY_DS_TEMP_2_REVISED   
"""

cur= ctx.cursor()
cur.execute(sql)
data = cur.fetch_pandas_all()

In [4]:
data.shape

(323507, 10)

In [180]:
sql_consider_cust = """ 

SELECT * FROM MRCH_SANDBOX.MRCH_DS_LOYALTY_CUST_SEGMENT
where CUSTOMER_SEGMENT = 'Not Assigned yet'
"""

cur= ctx.cursor()
cur.execute(sql_consider_cust)
data_consider = cur.fetch_pandas_all()
data_consider.shape

(281331, 5)

In [224]:
filtered_data = data[data["CUSTOMER_ID"].isin(data_consider["CUSTOMER_ID"].unique())]
merge_data = data[data["CUSTOMER_ID"].isin(data_consider["CUSTOMER_ID"].unique())]
filtered_data.shape

(281331, 10)

In [225]:
filtered_data["MC_FLAG"] = 0
filtered_data.loc[filtered_data["PERC_CHEWY_SALES"] > 0  , "MC_FLAG" ] = 1


merge_data["MC_FLAG"] = 0
merge_data.loc[merge_data["PERC_CHEWY_SALES"] > 0  , "MC_FLAG" ] = 1

filtered_data["concat"] = filtered_data['AS_FLAG'].astype(str) + '_'  + filtered_data['PET_FLAG'].astype(str) + '_'  + filtered_data['MC_FLAG'].astype(str)
merge_data["concat"] = merge_data['AS_FLAG'].astype(str) + '_'  + merge_data['PET_FLAG'].astype(str) + '_'  + merge_data['MC_FLAG'].astype(str)


filtered_data['COUNT'] = filtered_data.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT'])['CUSTOMER_ID'].transform('count')
filtered_data["combo_COUNT"] = filtered_data.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT' , 'concat'])['CUSTOMER_ID'].transform('nunique')





In [226]:
data.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT'])['CUSTOMER_ID'].count().sort_values()

REGULAR_CUSTOMER  QUARTILE  COHORT  
0                 3         COHORT_2     2143
                  1         COHORT_2     2144
                  4         COHORT_2     2144
                  2         COHORT_2     2145
1                 2         COHORT_2    14320
                  4         COHORT_2    14324
                  3         COHORT_2    14325
                  1         COHORT_2    14329
                  3         COHORT_1    20631
                  4         COHORT_1    20638
                  2         COHORT_1    20639
                  1         COHORT_1    20641
0                 2         COHORT_1    43739
                  3         COHORT_1    43769
                  1         COHORT_1    43781
                  4         COHORT_1    43795
Name: CUSTOMER_ID, dtype: int64

In [227]:
filtered_data.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT'])['CUSTOMER_ID'].count().sort_values()


REGULAR_CUSTOMER  QUARTILE  COHORT  
1                 2         COHORT_2    12320
                  4         COHORT_2    12324
                  3         COHORT_2    12325
                  1         COHORT_2    12329
                  3         COHORT_1    17431
                  4         COHORT_1    17438
                  2         COHORT_1    17439
                  1         COHORT_1    17441
0                 2         COHORT_1    40539
                  3         COHORT_1    40569
                  1         COHORT_1    40581
                  4         COHORT_1    40595
Name: CUSTOMER_ID, dtype: int64

In [228]:
filtered_data.shape[0]

281331

In [229]:
filtered_data["CUSTOMER_PROP"] = filtered_data['COUNT'] / filtered_data.shape[0]
filtered_data.loc[filtered_data["COHORT"] == 'COHORT_1' , "CUSTOMER_REQD"] = filtered_data["CUSTOMER_PROP"] * 36000
filtered_data.loc[filtered_data["COHORT"] == 'COHORT_2' , "CUSTOMER_REQD"] = filtered_data["CUSTOMER_PROP"] * 16000


filtered_Data = filtered_data[filtered_data["combo_COUNT"] >=3]

WAVE 2


In [230]:
selected_data = pd.DataFrame()

In [231]:
for regular in  [1]:
    for quartile in filtered_Data["QUARTILE"].unique():

        data_sel = filtered_Data[(filtered_Data["REGULAR_CUSTOMER"] == regular) & (filtered_Data["COHORT"] == "COHORT_1") & (filtered_Data["QUARTILE"] == quartile)]
 
        X_train, Test = train_test_split(data_sel , test_size= 4500 ,  stratify = data_sel['concat'])
        Test["Flag"] = "Test"

        last_split , Control = train_test_split(X_train , test_size = 4500 , stratify = X_train['concat'])
        Control["Flag"] = "Control" 


        temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  Control[["CUSTOMER_ID" , 'Flag']]  ])   
        selected_data = pd.concat([selected_data, temp_df])


In [232]:
for regular in  [0]:
    for quartile in filtered_Data["QUARTILE"].unique():

        data_sel = filtered_Data[(filtered_Data["REGULAR_CUSTOMER"] == regular) & (filtered_Data["COHORT"] == "COHORT_1") & (filtered_Data["QUARTILE"] == quartile)]
 
        X_train, Test = train_test_split(data_sel , test_size= 6500 ,  stratify = data_sel['concat'])
        Test["Flag"] = "Test"

        last_split , Control = train_test_split(X_train , test_size = 6500 , stratify = X_train['concat'])
        Control["Flag"] = "Control" 
        # X_train["Flag"] = "Test" 

        temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  Control[["CUSTOMER_ID" , 'Flag']]  ])   
        selected_data = pd.concat([selected_data, temp_df])
            

In [233]:
for regular in  [1]:
    for quartile in filtered_Data["QUARTILE"].unique():

        data_sel = filtered_Data[(filtered_Data["REGULAR_CUSTOMER"] == regular) & (filtered_Data["COHORT"] == "COHORT_2") & (filtered_Data["QUARTILE"] == quartile)]
 
        X_train, Test = train_test_split(data_sel , test_size= 4000 ,  stratify = data_sel['concat'])
        Test["Flag"] = "Test"

        last_split , Control = train_test_split(X_train , test_size = 4000 , stratify = X_train['concat'])
        Control["Flag"] = "Control" 


        temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  Control[["CUSTOMER_ID" , 'Flag']]  ])   
        selected_data = pd.concat([selected_data, temp_df])

In [234]:
selected_data.groupby("Flag")["CUSTOMER_ID"].count()

Flag
Control    60000
Test       60000
Name: CUSTOMER_ID, dtype: int64

In [235]:
merge_data_1 = merge_data.merge(selected_data , on = "CUSTOMER_ID" , how = 'left')
merge_data_1 ["Flag"].fillna("Not Assigned yet" , inplace = True)
wave2  = merge_data_1[merge_data_1["Flag"]!= "Not Assigned yet"]

In [236]:
test  = merge_data_1[merge_data_1["Flag"]!= "Not Assigned yet"].groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT', 'Flag']).agg({"CUSTOMER_ID" : 'count' , 'MC_FLAG' : 'sum' , 
                                                                                                                               'TENURE' : 'mean' , 'AS_FLAG' : 'sum'}).reset_index()
test["AS_PERC"] = test["AS_FLAG"] * 100 / test["CUSTOMER_ID"]

In [237]:
test.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT' ])['CUSTOMER_ID'].sum().sort_values()

REGULAR_CUSTOMER  QUARTILE  COHORT  
1                 1         COHORT_2     8000
                  2         COHORT_2     8000
                  3         COHORT_2     8000
                  4         COHORT_2     8000
                  1         COHORT_1     9000
                  2         COHORT_1     9000
                  3         COHORT_1     9000
                  4         COHORT_1     9000
0                 1         COHORT_1    13000
                  2         COHORT_1    13000
                  3         COHORT_1    13000
                  4         COHORT_1    13000
Name: CUSTOMER_ID, dtype: int64

In [238]:
test2 = merge_data_1[merge_data_1["Flag"]== "Not Assigned yet"].groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT', 'Flag']).agg({"CUSTOMER_ID" : 'count' , 'MC_FLAG' : 'sum' , 'TENURE' : 'mean' , 'AS_FLAG' : 'sum'}).reset_index()
test2["AS_PERC"] = test2["AS_FLAG"] * 100 / test2["CUSTOMER_ID"]

In [239]:
test2.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT' ])['CUSTOMER_ID'].sum().sort_values()

REGULAR_CUSTOMER  QUARTILE  COHORT  
1                 2         COHORT_2     4320
                  4         COHORT_2     4324
                  3         COHORT_2     4325
                  1         COHORT_2     4329
                  3         COHORT_1     8431
                  4         COHORT_1     8438
                  2         COHORT_1     8439
                  1         COHORT_1     8441
0                 2         COHORT_1    27539
                  3         COHORT_1    27569
                  1         COHORT_1    27581
                  4         COHORT_1    27595
Name: CUSTOMER_ID, dtype: int64

In [269]:
8000/(9000+13000)

0.36363636363636365

WAVE 3

In [257]:
wave3 = merge_data_1[merge_data_1["Flag"]== "Not Assigned yet"]
wave3.drop(columns = ["Flag"] , inplace = True)

In [258]:
selected_data = pd.DataFrame()

In [259]:
for regular in  [1]:
    for quartile in wave3["QUARTILE"].unique():

        data_sel = wave3[(wave3["REGULAR_CUSTOMER"] == regular) & (wave3["COHORT"] == "COHORT_1") & (wave3["QUARTILE"] == quartile)]
 
        Control, Test = train_test_split(data_sel , test_size= 0.5 ,  stratify = data_sel['concat'])
        Test["Flag"] = "Test"

        # last_split , Control = train_test_split(X_train , test_size = 4500 , stratify = X_train['concat'])
        Control["Flag"] = "Control" 


        temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  Control[["CUSTOMER_ID" , 'Flag']]  ])   
        selected_data = pd.concat([selected_data, temp_df])

In [260]:
for regular in  [0]:
    for quartile in wave3["QUARTILE"].unique():

        data_sel = wave3[(wave3["REGULAR_CUSTOMER"] == regular) & (wave3["COHORT"] == "COHORT_1") & (wave3["QUARTILE"] == quartile)]
 
        Control, Test = train_test_split(data_sel , test_size= 0.5 ,  stratify = data_sel['concat'])
        Test["Flag"] = "Test"
        Control["Flag"] = "Control" 

        temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  Control[["CUSTOMER_ID" , 'Flag']]  ])   
        selected_data = pd.concat([selected_data, temp_df])
            

In [261]:
for regular in  [1]:
    for quartile in wave3["QUARTILE"].unique():

        data_sel = wave3[(wave3["REGULAR_CUSTOMER"] == regular) & (wave3["COHORT"] == "COHORT_2") & (wave3["QUARTILE"] == quartile)]
 
        Control, Test = train_test_split(data_sel , test_size= 0.5 ,  stratify = data_sel['concat'])
        Test["Flag"] = "Test"
        Control["Flag"] = "Control" 


        temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  Control[["CUSTOMER_ID" , 'Flag']]  ])   
        selected_data = pd.concat([selected_data, temp_df])

In [262]:
wave3_merged = wave3.merge(selected_data , on = "CUSTOMER_ID" , how = 'left')


In [264]:
test  = wave3_merged.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT', 'Flag']).agg({"CUSTOMER_ID" : 'count' , 'MC_FLAG' : 'sum' , 
                                                                                                                               'TENURE' : 'mean' , 'AS_FLAG' : 'sum'}).reset_index()
test["AS_PERC"] = test["AS_FLAG"] * 100 / test["CUSTOMER_ID"]

In [271]:
test

,REGULAR_CUSTOMER,QUARTILE,COHORT,Flag,CUSTOMER_ID,MC_FLAG,TENURE,AS_FLAG,AS_PERC
0,0,1,COHORT_1,Control,13790,2170,1579.629877,6837,49.579405
1,0,1,COHORT_1,Test,13791,2169,1568.571967,6836,49.568559
2,0,2,COHORT_1,Control,13769,2451,1509.090130,8623,62.626189
3,0,2,COHORT_1,Test,13770,2447,1514.740741,8623,62.621641
4,0,3,COHORT_1,Control,13784,2691,1478.206544,10007,72.598665
5,0,3,COHORT_1,Test,13785,2690,1469.628292,10010,72.615161
6,0,4,COHORT_1,Control,13797,2551,1485.462709,9406,68.174241
7,0,4,COHORT_1,Test,13798,2548,1484.446369,9407,68.176547
8,1,1,COHORT_1,Control,4220,800,1554.098578,1860,44.075829
9,1,1,COHORT_1,Test,4221,798,1570.264629,1863,44.136461


In [270]:
test.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT', 'Flag' ])['CUSTOMER_ID'].sum().sort_values()

REGULAR_CUSTOMER  QUARTILE  COHORT    Flag   
1                 2         COHORT_2  Test        2160
                                      Control     2160
                  4         COHORT_2  Test        2162
                  3         COHORT_2  Control     2162
                  4         COHORT_2  Control     2162
                  3         COHORT_2  Test        2163
                  1         COHORT_2  Control     2164
                                      Test        2165
                  3         COHORT_1  Control     4215
                                      Test        4216
                  2         COHORT_1  Control     4219
                  4         COHORT_1  Control     4219
                                      Test        4219
                  1         COHORT_1  Control     4220
                  2         COHORT_1  Test        4220
                  1         COHORT_1  Test        4221
0                 2         COHORT_1  Control    13769
                   

In [267]:
8431+27595

36026

In [268]:
4320/36000

0.12

In [274]:
wave2["WAVE"] = 'WAVE2'
wave3_merged["WAVE"] = 'WAVE3'
final_data = pd.concat([wave2, wave3_merged])

In [279]:
final_data.groupby([ 'WAVE' , 'REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT' ])['CUSTOMER_ID'].count().sort_values()

WAVE  REGULAR_CUSTOMER  QUARTILE  COHORT  
3     1                 2         COHORT_2     4320
                        4         COHORT_2     4324
                        3         COHORT_2     4325
                        1         COHORT_2     4329
2     1                 3         COHORT_2     8000
                        2         COHORT_2     8000
                        4         COHORT_2     8000
                        1         COHORT_2     8000
3     1                 3         COHORT_1     8431
                        4         COHORT_1     8438
                        2         COHORT_1     8439
                        1         COHORT_1     8441
2     1                 3         COHORT_1     9000
                        1         COHORT_1     9000
                        4         COHORT_1     9000
                        2         COHORT_1     9000
      0                 2         COHORT_1    13000
                        3         COHORT_1    13000
                     

In [281]:
final_data=final_data[["REGULAR_CUSTOMER" , "COHORT" , "QUARTILE" ,  "CUSTOMER_ID" ,"Flag" , "WAVE" ]]

In [283]:
final_data.to_csv("remaining_wave.csv")